In [1]:
%matplotlib inline


A Gentle Introduction to ``torch.autograd``
---------------------------------

``torch.autograd`` is PyTorch’s automatic differentiation engine that powers neural network training. In this section, you will get a conceptual understanding of how autograd helps a neural network train.

``torch.autograd`` — это механизм автоматического дифференцирования PyTorch, который обеспечивает обучение нейронных сетей. В этом разделе вы получите концептуальное представление о том, как autograd помогает обучать нейронную сеть.

### Background

Neural networks (NNs) are a collection of nested functions that are executed on some input data. These functions are defined by *parameters* (consisting of weights and biases), which in PyTorch are stored in tensors.

Нейронные сети (НС) представляют собой набор вложенных функций, которые выполняются на некоторых входных данных. Эти функции определяются *параметрами* (состоящими из весов и смещений), которые в PyTorch хранятся в тензорах.

Training a NN happens in two steps (Обучение НС происходит в два этапа):

**Forward Propagation**: In forward prop, the NN makes its best guess about the correct output. It runs the input data through each of its functions to make this guess.

**Прямое распространение**: NN делает максимально возможное предположение о правильном выходе. Чтобы сделать это предположение, он пропускает входные данные через каждую из своих функций.

**Backward Propagation**: In backprop, the NN adjusts its parameters proportionate to the error in its guess. It does this by traversing backwards from the output, collecting the derivatives of the error with respect to the parameters of the functions (*gradients*), and optimizing the parameters using gradient descent. For a more detailed walkthrough of backprop, check out this video from 3Blue1Brown <https://www.youtube.com/watch?v=tIeHLnjs5U8>.

**Обратное распространение**: при обратном распространении NN корректирует свои параметры пропорционально ошибке в своем предположении. Он делает это, проходя назад от выходных данных, собирая производные ошибки по параметрам функций (*градиенты*) и оптимизируя параметры с помощью градиентного спуска. Более подробное описание backprop можно посмотреть в этом видео от 3Blue1Brown <https://www.youtube.com/watch?v=tIeHLnjs5U8>.

### Usage in PyTorch

Let's take a look at a single training step.
For this example, we load a pretrained resnet18 model from ``torchvision``.
We create a random data tensor to represent a single image with 3 channels, and height & width of 64, and its corresponding ``label`` initialized to some random values.

Давайте рассмотрим один этап обучения.
В этом примере мы загружаем предварительно обученную модель resnet18 из torchvision.
Мы создаем тензор случайных данных для представления одного изображения с тремя каналами, высотой и шириной 64, а также соответствующую ему «метку», инициализированную некоторыми случайными значениями.

In [2]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

C:\Users\iveel\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\iveel\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Next, we run the input data through the model through each of its layers to make a prediction.
This is the **forward pass**.

Затем мы пропускаем входные данные через модель через каждый из ее слоев, чтобы сделать прогноз


In [3]:
prediction = model(data) # forward pass

In [4]:
sum(p.numel() for p in model.parameters())

11689512

In [5]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Это архитектура **ResNet** (Residual Network), предложенная в статье "Deep Residual Learning for Image Recognition" и используемая для классификации изображений.

**Давайте разберем каждую часть этой архитектуры:**

`conv1`: это первый сверточный слой (Conv2d), который принимает изображения с тремя каналами (RGB). Он использует 64 фильтра размером 7x7 с шагом 2 и нулевым заполнением (padding), что уменьшает размер изображения вдвое (stride=2).
Аргумент bias=False указывает, что в слое отсутствуют смещения.

`bn1`: это слой нормализации пакета (BatchNorm2d), который нормализует активации после первого сверточного слоя. `momentum=0.1` означает, что статистика пакета (например, среднее и дисперсия) обновляется в слое нормализации пакета во время обучения относительно медленно, что может увеличить стабильность обучения. Если `affine=True`, то слой будет иметь два набора параметров (масштаб и сдвиг), которые могут использоваться для масштабирования и сдвига нормализованных значений. Если `affine=False`, то слой не будет иметь параметры масштабирования и сдвига. Если `track_running_stats=True`, то слой будет отслеживать и использовать «накопленные» статистические данные во время инференса (процесс классификации или предсказания) для нормализации данных.

`relu`: это функция активации ReLU (ReLU), которая применяется после слоя нормализации. Когда ``inplace=True``, функция активации ReLU изменяет исходный тензор, а не создает новый тензор для выходных данных. Это может сэкономить память и ускорить выполнение, поскольку не требуется дополнительное выделение памяти под новый тензор.

`maxpool`: это слой максимальной подвыборки (MaxPool2d), который выполняет максимальную подвыборку с размером окна 3x3 и шагом 2, что уменьшает размер изображения. ``padding`` указывает на количество нулевых пикселей, которые окружают входное изображение (или выход из предыдущего слоя) до применения сверточного ядра. В данном случае, ``padding=1`` означает, что вокруг каждой стороны изображения будет добавлен один пиксель с нулевым значением. Это обычно используется для сохранения размера изображения после свертки. `dilation` (расширение) определяет интервал между пикселями в сверточном ядре. Значение по умолчанию 1 означает, что пиксели в ядре располагаются рядом друг с другом. Увеличение `dilation` приведет к более далекому размещению пикселей в ядре, что может увеличить размер поля видимости свертки. ``ceil_mode`` определяет, должна ли свертка "округлять вверх" размеры выхода при использовании сверточного слоя с заданным stride. Если ``ceil_mode=True``, то размер выхода округляется вверх до ближайшего целого числа, чтобы обеспечить точное соответствие размерам входа и выхода. Если ``ceil_mode=False`` (значение по умолчанию), то размер выхода округляется вниз, что может привести к небольшому уменьшению размера выхода.

`layer1, layer2, layer3, и layer4`: это последовательности (Sequential) блоков BasicBlock, которые представляют собой основные строительные блоки ResNet. Каждый BasicBlock состоит из двух сверточных слоев, слоев нормализации и функции активации. Блоки в layer1, layer2, layer3 и layer4 отличаются количеством фильтров и размером ядер в сверточных слоях.
Они также могут содержать блок `downsample` в случае, если размерность входных данных изменяется. Этот блок используется для выравнивания размеров входных данных и выходных данных.

`avgpool`: это слой адаптивной средней подвыборки (AdaptiveAvgPool2d), который усредняет значения по всем пикселям, чтобы получить фиксированный размер выхода (1x1).

`fc`: это полносвязный слой (Linear), который используется для классификации. В данном случае, он имеет 1000 выходных нейронов, что может использоваться для классификации в 1000 классов. Это часто используется в задачах ImageNet.

We use the model's prediction and the corresponding label to calculate the error (``loss``).
The next step is to backpropagate this error through the network.
Backward propagation is kicked off when we call ``.backward()`` on the error tensor.
Autograd then calculates and stores the gradients for each model parameter in the parameter's ``.grad`` attribute.

Мы используем прогноз модели и соответствующую метку для расчета ошибки (``loss``).
Следующим шагом является обратное распространение этой ошибки по сети.
Обратное распространение запускается, когда мы вызываем ``.backward()`` для тензора ошибок.
Затем Autograd вычисляет и сохраняет градиенты для каждого параметра модели в атрибуте .grad параметра.


In [6]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Next, we load an optimizer, in this case SGD with a learning rate of 0.01 and momentum of 0.9.
We register all the parameters of the model in the optimizer.

Затем мы загружаем оптимизатор, в данном случае SGD, со скоростью обучения 0,01 и импульсом 0,9.
Прописываем все параметры модели в оптимизаторе.

In [7]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Finally, we call ``.step()`` to initiate gradient descent. The optimizer adjusts each parameter by its gradient stored in ``.grad``.

Наконец, мы вызываем ``.step()``, чтобы инициировать градиентный спуск. Оптимизатор настраивает каждый параметр по его градиенту, хранящемуся в ``.grad``.

In [8]:
optim.step() #gradient descent

At this point, you have everything you need to train your neural network.
The below sections detail the workings of autograd - feel free to skip them.

На этом этапе у вас есть все необходимое для обучения нейронной сети.

# Differentiation in Autograd

Let's take a look at how ``autograd`` collects gradients. We create two tensors ``a`` and ``b`` with
``requires_grad=True``. This signals to ``autograd`` that every operation on them should be tracked.

Давайте посмотрим, как ``autograd`` собирает градиенты. Мы создаем два тензора ``a`` и ``b`` с помощью
``requires_grad=True``. Это сигнализирует autograd, что каждая операция над ними должна отслеживаться.

In [18]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

We create another tensor ``Q`` from ``a`` and ``b``.

\begin{align}Q = 3a^3 - b^2\end{align}



In [19]:
Q = 3*a**3 - b**2

In [11]:
Q.backward(torch.tensor([1, 1]))

In [12]:
a.grad, b.grad

(tensor([36., 81.]), tensor([-12.,  -8.]))

In [14]:
Q.sum().backward()

Let's assume ``a`` and ``b`` to be parameters of an NN, and ``Q``
to be the error. In NN training, we want gradients of the error
w.r.t. parameters (градиенты ошибок относительно параметров), i.e.

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}


When we call ``.backward()`` on ``Q``, autograd calculates these gradients and stores them in the respective tensors' ``.grad`` attribute.

Когда мы вызываем .backward() для Q, autograd вычисляет эти градиенты и сохраняет их в атрибуте .grad соответствующего тензора.

We need to explicitly pass a ``gradient`` argument in ``Q.backward()`` because it is a vector. ``gradient`` is a tensor of the same shape as ``Q``, and it represents the gradient of Q w.r.t. itself, i.e.

Нам нужно явно передать аргумент градиента в Q.backward(), потому что это вектор. ``градиент`` — это тензор той же формы, что и ``Q``, и он представляет собой градиент Q относительно самого себя, т.е.

\begin{align}\frac{dQ}{dQ} = 1\end{align}

Equivalently, we can also aggregate Q into a scalar and call backward implicitly, like ``Q.sum().backward()``.

Аналогично, мы также можем агрегировать Q в скаляр и неявно вызывать обратный вызов, например ``Q.sum().backward()``.

In [20]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradients are now deposited in ``a.grad`` and ``b.grad``



In [21]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


# Vector Calculus using ``autograd``

Mathematically, if you have a vector valued function (вектор-функция)
$\vec{y}=f(\vec{x})$, then the gradient of $\vec{y}$ with
respect to $\vec{x}$ is a Jacobian matrix $J$:

\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector $\vec{v}$, compute the product
$J^{T}\cdot \vec{v}$

If $\vec{v}$ happens to be the gradient of a scalar function $l=g\left(\vec{y}\right)$:

\begin{align}\vec{v}
   =
   \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}\end{align}

then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)\end{align}

This characteristic of vector-Jacobian product is what we use in the above example;
``external_grad`` represents $\vec{v}$.




# Computational Graph

Conceptually, autograd keeps a record of data (tensors) & all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of Function <https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function> objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

Концептуально, autograd хранит запись данных (тензоров) и всех выполненных операций (вместе с результирующими новыми тензорами) в направленном ациклическом графе (DAG), состоящем из объектов-функций. В этом DAG листья являются входными тензорами, корни — выходными тензорами. Проследив этот график от корней до листьев, вы можете автоматически вычислить градиенты, используя правило дифференцирования сложной функции.

In a **forward pass**, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor, and
- maintain the operation’s *gradient function* in the DAG.

При прямом проходе autograd одновременно делает две вещи:

- запускает запрошенную операцию для вычисления результирующего тензора и
- поддерживает *градиентную функцию* операции в DAG.

The **backward pass** kicks off when ``.backward()`` is called on the DAG root. ``autograd`` then:

- computes the gradients from each ``.grad_fn``,
- accumulates them in the respective tensor’s ``.grad`` attribute, and
- using the chain rule, propagates all the way to the leaf tensors.

Обратный проход начинается, когда в корне DAG вызывается .backward(), тогда ``autograd``:

- вычисляет градиенты для каждого ``.grad_fn``,
- накапливает их в атрибуте ``.grad`` соответствующего тензора, и
- используя chain rule, распространяется вплоть до листовых тензоров.

Below is a visual representation of the DAG in our example. In the graph, the arrows are in the direction of the forward pass. The nodes represent the backward functions of each operation in the forward pass. The leaf nodes in blue represent our leaf tensors ``a`` and ``b``.

figure:: /_static/img/dag_autograd.png

<div class="alert alert-info"><h4>Note</h4><p>DAGs are dynamic in PyTorch</p>
  <p>An important thing to note is that the graph is recreated from scratch; after each ``.backward()`` call, autograd starts populating a new graph. This is exactly what allows you to use control flow statements in your model; you can change the shape, size and operations at every iteration if needed.</p></div>

Exclusion from the DAG

``torch.autograd`` tracks operations on all tensors which have their ``requires_grad`` flag set to ``True``. For tensors that don’t require gradients, setting this attribute to ``False`` excludes it from the gradient computation DAG.

The output tensor of an operation will require gradients even if only a single input tensor has ``requires_grad=True``.

Исключение из DAG

``torch.autograd`` отслеживает операции со всеми тензорами, для которых флаг ``require_grad`` установлен в значение ``True``. Для тензоров, которые не требуют градиентов, установка для этого атрибута значения ``False`` исключает его из ациклического графа, ориентированного на вычисление градиента.

Выходной тензор операции потребует градиены, даже если только один входной тензор имеет ``require_grad=True``.

In [22]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


In a NN, parameters that don't compute gradients are usually called **frozen parameters**.
It is useful to "freeze" part of your model if you know in advance that you won't need the gradients of those parameters
(this offers some performance benefits by reducing autograd computations).

Another common usecase where exclusion from the DAG is important is for
`finetuning a pretrained network` <https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html>

In finetuning, we freeze most of the model and typically only modify the classifier layers to make predictions on new labels.
Let's walk through a small example to demonstrate this. As before, we load a pretrained resnet18 model, and freeze all the parameters.

В нейронной сети параметры, которые не вычисляют градиенты, обычно называются **замороженными параметрами**.
Полезно «заморозить» часть вашей модели, если вы заранее знаете, что вам не понадобятся градиенты этих параметров (это дает некоторые преимущества в производительности за счет сокращения autograd вычислений).

Другой распространенный случай использования, когда исключение из DAG важно, — это тонкая настройка предварительно обученной сети <https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html>

При точной настройке мы замораживаем большую часть модели и обычно изменяем только слои классификатора, чтобы делать прогнозы для новых меток.
Давайте рассмотрим небольшой пример, чтобы продемонстрировать это. Как и раньше, мы загружаем предварительно обученную модель resnet18 и закрепляем все параметры.

In [23]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Let's say we want to finetune the model on a new dataset with 10 labels.
In resnet, the classifier is the last linear layer ``model.fc``.
We can simply replace it with a new linear layer (unfrozen by default) that acts as our classifier.

Допустим, мы хотим точно настроить модель на новом наборе данных с 10 метками.
В Resnet классификатором является последний линейный слой `model.fc`.
Мы можем просто заменить его новым линейным слоем (по умолчанию незамороженным), который действует как наш классификатор.

In [24]:
model.fc = nn.Linear(512, 10)

Now all parameters in the model, except the parameters of ``model.fc``, are frozen.
The only parameters that compute gradients are the weights and bias of ``model.fc``.

Теперь все параметры модели, кроме параметров `model.fc`, заморожены.
Единственные параметры, которые вычисляют градиенты, — это веса и смещение `model.fc`.

In [25]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Notice although we register all the parameters in the optimizer, the only parameters that are computing gradients (and hence updated in gradient descent) are the weights and bias of the classifier.

The same exclusionary functionality is available as a context manager in `torch.no_grad()` <https://pytorch.org/docs/stable/generated/torch.no_grad.html>

Обратите внимание, что хотя мы регистрируем все параметры в оптимизаторе, единственными параметрами, которые вычисляют градиенты (и, следовательно, обновляются при градиентном спуске), являются веса и смещение классификатора.

Та же исключительная функциональность доступна в качестве менеджера контекста в `torch.no_grad()`
<https://pytorch.org/docs/stable/generated/torch.no_grad.html>

--------------




Further readings:
~~~~~~~~~~~~~~~~~~~

-  `In-place operations & Multithreaded Autograd <https://pytorch.org/docs/stable/notes/autograd.html>`__
-  `Example implementation of reverse-mode autodiff <https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC>`__

